<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/hurr/ATL/2025/combine_forecast_incl_IC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime, timedelta

import glob as glob
import xarray as xr

In [ ]:
date_start, date_end = ['20251020', '20251030']

save_data_path = '/content/drive/MyDrive/RTOFS/publications/Melissa2025_nATL/saved_data/RTOFS/'

## Add initial time (from nowcast) to the a single dataset

In [ ]:
start_date = datetime.strptime(date_start, '%Y%m%d')
end_date = datetime.strptime(date_end, '%Y%m%d')

current_date = start_date
while current_date <= end_date:
    date_str = current_date.strftime('%Y%m%d')
    fname_nowcast = f'{save_data_path}nowcast/{date_str}.nc'
    fname_forecast = f'{save_data_path}forecast/{date_str}.nc'

    ds_nowcast = xr.open_dataset(fname_nowcast)
    ds_forecast = xr.open_dataset(fname_forecast)

    ds_nowcast_filtered = ds_nowcast.sel(MT=ds_nowcast.MT.dt.hour == 0).isel(Depth=0)
    ds_forecast_filtered = ds_forecast.sel(MT=ds_forecast.MT.dt.hour == 0).isel(Depth=0)

    combined_ds = xr.concat([ds_nowcast_filtered.isel(MT=-1), ds_forecast_filtered], dim='MT')

    print(f'Processed data for {date_str}. Combined dataset has {combined_ds.sizes["MT"]} time steps.')

    fname_save = f'{save_data_path}combined/{date_str}.nc'
    combined_ds.to_netcdf(fname_save)
    print(f'Saved combined dataset to {fname_save}')

    current_date += timedelta(days=1)

## Plot

In [ ]:
#combined_ds.isel(Depth=0).sel(Y=slice(100, 375), X=slice(100, 500)).sel(MT=combined_ds.MT.dt.hour == 0).temperature.plot(x='X', y='Y', col='MT', vmin=24, vmax=32, cmap='Spectral_r')